<a href="https://colab.research.google.com/github/fjasic/duusaiuv/blob/main/vezba3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

In [7]:
!wget https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip
!wget https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip
!unzip -qq BelgiumTSC_Training.zip
!unzip -qq BelgiumTSC_Testing.zip

--2021-02-03 14:52:01--  https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip
Resolving btsd.ethz.ch (btsd.ethz.ch)... 129.132.52.185, 2001:67c:10ec:36c2::b9
Connecting to btsd.ethz.ch (btsd.ethz.ch)|129.132.52.185|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174298785 (166M) [application/zip]
Saving to: ‘BelgiumTSC_Training.zip’

BelgiumTSC_Training 100%[===================>] 166.22M   536KB/s    in 6m 18s  

2021-02-03 14:58:20 (450 KB/s) - ‘BelgiumTSC_Training.zip’ saved [174298785/174298785]

--2021-02-03 14:58:20--  https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip
Resolving btsd.ethz.ch (btsd.ethz.ch)... 129.132.52.185, 2001:67c:10ec:36c2::b9
Connecting to btsd.ethz.ch (btsd.ethz.ch)|129.132.52.185|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77877833 (74M) [application/zip]
Saving to: ‘BelgiumTSC_Testing.zip’

BelgiumTSC_Testing. 100%[===================>]  74.27M   506KB/s    in 2m 28s  

20

In [9]:
# Vežba 3 - konvolucioni slojevi
from __future__ import division, print_function, absolute_import

import numpy as np # linearna algebra
import pandas as pd # obrada podataka, csv datoteka (npr. pd.read_csv)

import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import skimage.transform

import tensorflow as tf

import random
import os


print(tf.__version__)
# Model:
#
# · · · · · · · · · ·      (ulazni podaci, dubina 3)            X [batch, 32, 32, 3]
# @ @ @ @ @ @ @ @ @ @   -- konv. sloj 5x5x3=>6, pomeraj 1       W1 [5, 5, 3, 6]        B1 [6]
# ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                           Y1 [batch, 32, 32, 6]
#   @ @ @ @ @ @ @ @     -- konv. sloj 5x5x6=>12, pomeraj 2      W2 [5, 5, 6, 12]       B2 [12]
#   ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                             Y2 [batch, 16, 16, 12]
#     @ @ @ @ @ @       -- konv. sloj 4x4x12=>24 pomeraj 2      W3 [4, 4, 12, 24]      B3 [24]
#     ∶∶∶∶∶∶∶∶∶∶∶                                               Y3 [batch, 8, 8, 24] => 'flattened' u YY [batch, 8*8*24]
#      \x/x\x\x/        -- potpuno povezani sloj (relu)         W4 [8*8*24, 200]       B4 [200]
#       · · · ·                                                 Y4 [batch, 200]
#       \x/x\x/         -- potpuno povezani sloj (softmax)      W5 [200, 62]           B5 [62]
#        · · ·                                                  Y [batch, 62]

train_data_dir = os.path.join("Training")
test_data_dir = os.path.join("Testing")

###########################################################################################
# Pomoćna funkcija za učitavanje skupa podataka
# load_data - učitava skup podataka i vraća dve liste: 
# - images: lista Numpy nizova, svaki predstavlja sliku.
# - labels: lista brojeva koji predstavljaju labele slika.
###########################################################################################
def load_data(data_dir):
    # Prikupljanje svih poddirektorijume direktorijuma data_dir. Svaki od njih predstavlja labelu.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Iteriranje kroz direktorijume (sa imenima labela) i prikupljanje podataka u dve liste: labele i slike.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) 
                      for f in os.listdir(label_dir) if f.endswith(".ppm")]
        # Za svaku labelu, učitavanje pripadajućih slika i dodavanje u listu slika.
        # I dodavanje labele (t.j. imena direktorijuma) u listu labela.
        for f in file_names:
            images.append(mpimg.imread(f))
            labels.append(int(d))
    return images, labels

###########################################################################################
# Učitavanje i predobrada trening skupa podataka
###########################################################################################
# Učitavanje trening skupa podataka.
images, labels = load_data(train_data_dir)

print("\nTrening skup podataka\n-Broj klasa/labela: {0}\n-Ukupan broj slika: {1}".format(len(set(labels)), len(images)))

# Za prvih 5 slika prikaz rezolucije slika (shape), kao i minimalne i maksimalne vrednosti niza (vrednost komponenti boja)
for image in images[:5]:
    print("\nOriginalni dataset: Oblik: {0}, min: {1}, max: {2}".format(image.shape, image.min(), image.max()))

# Skaliranje slika na istu rezoluciju (WxH -> 32x32)
scaled_imgs = [skimage.transform.resize(image, (32, 32), mode='constant')
                for image in images]

# Za prvih 5 slika prikaz rezolucije slika (shape), kao i minimalne i maksimalne vrednosti niza (vrednost komponenti boja)
for image in scaled_imgs[:5]:
    print("\nSkalirani dataset: Oblik: {0}, min: {1}, max: {2}".format(image.shape, image.min(), image.max()))

# Promena formata skladištenja: lista -> niz
labels_a = np.array(labels)         # lista -> (4575,)
images_a = np.array(scaled_imgs)    # lista -> (4575, 32, 32, 3)
# Prikaz 
print("\nRezultat predobrade:\n-Oblik labela: ", labels_a.shape, "\n-Oblik slika: ", images_a.shape)

1.15.2

Trening skup podataka
-Broj klasa/labela: 62
-Ukupan broj slika: 4575

Originalni dataset: Oblik: (151, 156, 3), min: 13, max: 255

Originalni dataset: Oblik: (94, 106, 3), min: 3, max: 255

Originalni dataset: Oblik: (60, 63, 3), min: 4, max: 255

Originalni dataset: Oblik: (319, 216, 3), min: 5, max: 255

Originalni dataset: Oblik: (151, 160, 3), min: 2, max: 255

Skalirani dataset: Oblik: (32, 32, 3), min: 0.09019607843137255, max: 0.996078431372549

Skalirani dataset: Oblik: (32, 32, 3), min: 0.06789215686274504, max: 0.996078431372549

Skalirani dataset: Oblik: (32, 32, 3), min: 0.08044194240196023, max: 0.996078431372549

Skalirani dataset: Oblik: (32, 32, 3), min: 0.048874080882352636, max: 0.996078431372549

Skalirani dataset: Oblik: (32, 32, 3), min: 0.03866421568627451, max: 0.9921568627450981

Rezultat predobrade:
-Oblik labela:  (4575,) 
-Oblik slika:  (4575, 32, 32, 3)


In [21]:
###########################################################################################
# Kreiranje modela
###########################################################################################
K = 6   # prvi konvolucioni sloj - dubina izlaza
L = 12  # drugi konvolucioni sloj dubina izlaza
M = 24  # treći konvolucioni sloj - dubina izlaza
N = 200  # broj neurona prvog potpuno povezanog sloja
num_classes = 62  # broj neurona izlaznog sloja = broj klasa

# Kreirati varijable za težinske koeficijente (W1,B1,W2,B2,W3,B3,W4,B4,W5,B5)
# -> Inicijalizovati koeficijente malim nasumičnim vrednostima između -0.2 i +0.2
# -> Prilikom korišćenja RELU aktivacione funkcije, bias koeficijenti moraju biti male *pozitivne* vrednosti (npr. 0.1)
W1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 3, K], stddev=0.1), name = "W1")      
B1 = tf.Variable(tf.ones([K])/10, name = "B1") 

W2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 6, L], stddev=0.1), name = "W2")      
B2 = tf.Variable(tf.ones([L])/10, name = "B2") 

W3 = tf.Variable(tf.truncated_normal(shape=[4, 4, 12, M], stddev=0.1), name = "W3")      
B3 = tf.Variable(tf.ones([M])/10, name = "B3") 

W4 = tf.Variable(tf.truncated_normal(shape=[8*8*24, N], stddev=0.1), name = "W4")      
B4 = tf.Variable(tf.ones([N])/10, name = "B4") 

W5 = tf.Variable(tf.truncated_normal(shape=[N, num_classes], stddev=0.1), name = "W5")      
B5 = tf.Variable(tf.ones([num_classes])/10, name = "B5")

# Kreirati rezervisane varijable (placeholders) za slike i labele.
# Koristiti None ya oblik, kako bi se veličina batch-a mogla menjati, nakon što s ekonstruiše graf
# -> ulaz X: 32x32x3 slike, prva dimenzija (None) indeksira slike unutar batch-a
images_ph = tf.placeholder(tf.float32, [None, 32, 32, 3])
labels_ph = tf.placeholder(tf.int32, [None])

# TODO -> Definisati model
# -> promeniti oblik(način smeštanja) izlaza iz trećeg konvolucionog sloja kako bi odgovarao ulazu za potpuno povezani sloj
sloj_1 = tf.nn.conv2d(input=images_ph, filter=W1, strides=[1, 1, 1, 1], padding="SAME", use_cudnn_on_gpu=True,
                      dilations=[1, 1, 1, 1], name=None) + B1

sloj_2 = tf.nn.conv2d(input=sloj_1, filter=W2, strides=[1, 2, 2, 1], padding="SAME", use_cudnn_on_gpu=True,
                      dilations=[1, 1, 1, 1], name=None ) + B2

sloj_3 = tf.nn.conv2d(input=sloj_2, filter=W3, strides=[1, 2, 2, 1], padding="SAME", use_cudnn_on_gpu=True,
                      dilations=[1, 1, 1, 1], name=None ) + B3

flatten_sloj = tf.contrib.layers.flatten(sloj_3)
sloj_4 = tf.nn.relu(tf.matmul(flatten_sloj, W4) + B4)
logits = tf.matmul(sloj_4, W5) + B5

Y = tf.nn.softmax(logits)

# Odrediti klase na osnovu verovatnoća izlaznog sloja
predicted_classes = tf.argmax(Y, 1)

# Definisati funkciju mere gubitka
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels_ph))

# Definisati algoritam za adaptaciju koeficijenata i brzine učenja
train = tf.train.MomentumOptimizer(learning_rate=0.0005, momentum=0.9, use_nesterov=True).minimize(loss)

# Operacija inicijalizacije varijabli
init = tf.global_variables_initializer()

In [24]:
###########################################################################################
# Proces treniranja
###########################################################################################

# Podešavanje parametara 
num_epochs = 200   # broj epoha
num_batches = 25  # broj batch-eva u okviru  epohe
b_size = 183      # broj slika u batch-u

# Kreiranje sesije
session = tf.Session()
session.run(init)

# Za svaku epohu
for i in range(num_epochs):
    # I svaki batch u epohi
    for j in range(num_batches):
        _, loss_value = session.run([train, loss], 
                                    feed_dict={images_ph: images_a[j*b_size:(j+1)*b_size], labels_ph: labels_a[j*b_size:(j+1)*b_size]})
                                    
    if i % 10 == 0:
        # Izvršavanje "predicted_classes" čvora
        predicted = session.run([predicted_classes], feed_dict={images_ph: scaled_imgs})[0]

        # Izračunavanje tačnosti
        match_count = sum([int(y == y_) for y, y_ in zip(labels, predicted)])
        accuracy = match_count / len(labels)
        print("Trening podaci: Epoha {0}: Mera gubitka: {1}, Tačnost: {2}".format(i, loss_value, accuracy))


Trening podaci: Epoha 0: Mera gubitka: 4.490381717681885, Tačnost: 0.041530054644808745
Trening podaci: Epoha 10: Mera gubitka: 1.4075584411621094, Tačnost: 0.5665573770491803
Trening podaci: Epoha 20: Mera gubitka: 1.0362995862960815, Tačnost: 0.7586885245901639
Trening podaci: Epoha 30: Mera gubitka: 0.675529420375824, Tačnost: 0.8465573770491803
Trening podaci: Epoha 40: Mera gubitka: 0.440976619720459, Tačnost: 0.9027322404371585
Trening podaci: Epoha 50: Mera gubitka: 0.27310115098953247, Tačnost: 0.9250273224043716
Trening podaci: Epoha 60: Mera gubitka: 0.24970372021198273, Tačnost: 0.9027322404371585
Trening podaci: Epoha 70: Mera gubitka: 0.09246090054512024, Tačnost: 0.9702732240437159
Trening podaci: Epoha 80: Mera gubitka: 0.050676558166742325, Tačnost: 0.981639344262295
Trening podaci: Epoha 90: Mera gubitka: 0.031228922307491302, Tačnost: 0.9901639344262295
Trening podaci: Epoha 100: Mera gubitka: 0.0209694504737854, Tačnost: 0.993879781420765
Trening podaci: Epoha 110: M

In [25]:
###########################################################################################
# Evaluacija
###########################################################################################

# Učitavanje test skupa podataka
test_images, test_labels = load_data(test_data_dir)

# Predobrada slika, identično kao i za trening skup podataka.
test_scaled_imgs = [skimage.transform.resize(image, (32, 32), mode='constant')
                 for image in test_images]
test_labels_a = np.array(test_labels)
test_images_a = np.array(test_scaled_imgs)

# Pokretanje inferencije nad celim testnim skupom slika
predicted = session.run([predicted_classes], 
                        feed_dict={images_ph: test_scaled_imgs})[0]
 
test_loss_value = session.run(loss, 
                        feed_dict={images_ph: test_images_a, labels_ph: test_labels_a})

# Izračunavanje tačnosti
match_count = sum([int(y == y_) for y, y_ in zip(test_labels, predicted)])
test_accuracy = match_count / len(test_labels)
print("Test podaci: Mera gubitka: {0}, Tačnost: {1}".format(test_loss_value, test_accuracy))

session.close()

Test podaci: Mera gubitka: 0.9413958787918091, Tačnost: 0.8492063492063492
